In [1]:
import ITensors:op
import Pkg; Pkg.add("HDF5")
import Pkg; Pkg.add("Plots")
import Pkg; Pkg.add("PyFormattedStrings")
import Pkg; Pkg.add("LinearAlgebra")
import Pkg; Pkg.add("LaTeXStrings")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
using ITensors, ITensorMPS
import ITensors:op
using HDF5
using Plots
using PyFormattedStrings
using LinearAlgebra
using ITensorTDVP: tdvp

ITensors.disable_warn_order()

LoadError: ArgumentError: Package ITensorTDVP not found in current path.
- Run `import Pkg; Pkg.add("ITensorTDVP")` to install the ITensorTDVP package.

In [3]:
#include("overload_Spin=3h.jl")
# overloadings for SiteType 3/2
#----------------------------------------------
ITensors.space(::SiteType"S=3/2") = 4

# operator Sz
#-------------------
ITensors.op(::OpName"Sz",::SiteType"S=3/2") =
[+3/2   0    0    0
  0    +1/2  0    0
  0     0   -1/2  0
  0     0    0   -3/2]

# operator S+
#-------------------
ITensors.op(::OpName"S+",::SiteType"S=3/2") =
[0  √3  0   0 
 0  0   √4  0
 0  0   0   √3
 0  0   0   0]

# operator S-
#-------------------
ITensors.op(::OpName"S-",::SiteType"S=3/2") =
[0  0   0   0
 √3 0   0   0
 0  √4  0   0
 0  0   √3  0]

# z projection states
#-----------------------------------------
ITensors.state(::StateName"+3/2",::SiteType"S=3/2") = [1, 0, 0, 0]
ITensors.state(::StateName"+1/2",::SiteType"S=3/2") = [0, 1, 0, 0]
ITensors.state(::StateName"-1/2",::SiteType"S=3/2") = [0, 0, 1, 0]
ITensors.state(::StateName"-3/2",::SiteType"S=3/2") = [0, 0, 0, 1]

In [4]:
# construction of the MPO hamiltonian
#------------------------------------------
function Hamiltonian(sites)
    ops = OpSum()

    N = length(sites)
    # spin-spin interaction
    for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)
    #--------------------------
    return H
end

Hamiltonian (generic function with 1 method)

In [5]:
function Hamiltonian1(sites)
    ops = OpSum()

    N = length(sites)
    # spin-spin interaction
    for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:2
        ops += -B, "Sz",k
    end
    ops += -B1, "Sz",3
    
    for k=4:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)
    #--------------------------
    return H
end

Hamiltonian1 (generic function with 1 method)

In [6]:

#four point RDM of sites AB, CD
# AB adjacent, CD adjacent
#------------------------------------------
function get_4pdm(wf::MPS,locs)
    psi = copy(wf)
    A,C = locs

    orthogonalize!(psi,A)
    ket = psi[A]
    for k in A+1:C+1
        ket *= psi[k]
    end
    rho = prime(ket,"Site") * dag(ket)

    if C-A>2
        inds_list = inds(rho)
        trinds1 = collect(3 : C-A)
        trinds2 = trinds1[end] .+ trinds1 .+ 2
        for idx in 1:length(trinds1)
            rho = rho*delta(inds_list[trinds1[idx]],inds_list[trinds2[idx]])
        end
    end
    #-------------------
    return rho
end

get_4pdm (generic function with 1 method)

In [7]:
#   calc_Eigenvalues of density matrix
#-----------------------------------
function calc_Eigenvalues(density_matrix)
    egn_val,_ = eigen(density_matrix,ishermitian=true)
    #-----------------------------------
    return diag(array(egn_val))
end


#   eigenvalue based generic properties of a density matrix
#-----------------------------------
function calc_Norm(density_matrix)
    rho = copy(density_matrix)

    egn_val = calc_Eigenvalues(rho)
    Norm = sum(egn_val)
    Purity = sum(egn_val.^2)

    #-----------------------------------
    return [Norm,Purity]
end

#   eigenvalue based generic properties of a density matrix
#-----------------------------------
function calc_SvN(density_matrix)
    rho = copy(density_matrix)

    egn_val = calc_Eigenvalues(rho)
    SvN = sum( [ - lam*log(lam) for lam in egn_val if lam > 0 ] )

    #-----------------------------------
    return SvN
end


calc_SvN (generic function with 1 method)

In [8]:
#define the parameters of the hamiltonian and constuct MPO
#----------------------------------------------------------------------------
J = 1 
B = 0.1
B1 = 0.001
nsweeps = 10
maxdim = [100, 200, 500, 1000, 2000, 5000, 10000, 25580]
cutoff = [1E-11]

#--------------------
N = 14
#--------------------

site_list = collect(1:N) 
sites = siteinds( k->isodd(k) ? "S=1/2" : "S=3/2", N ) 
#sites = siteinds("S=1/2", N)

H = Hamiltonian(sites) 

# ground state and first excited state
#--------------------------------------------
energy0,psi0 = dmrg(H,random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;
#energy1,psi1 = dmrg(H,[psi0],random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;

After sweep 1 energy=-13.78403323841318  maxlinkdim=8 maxerr=2.84E-16 time=22.189
After sweep 2 energy=-14.453456453948444  maxlinkdim=32 maxerr=2.12E-16 time=0.245
After sweep 3 energy=-14.470386556944907  maxlinkdim=110 maxerr=9.49E-12 time=0.764
After sweep 4 energy=-14.470633004996241  maxlinkdim=156 maxerr=9.99E-12 time=1.443
After sweep 5 energy=-14.470654792637209  maxlinkdim=122 maxerr=9.98E-12 time=1.664
After sweep 6 energy=-14.470657805398277  maxlinkdim=95 maxerr=9.93E-12 time=0.971
After sweep 7 energy=-14.47065831073599  maxlinkdim=77 maxerr=9.80E-12 time=0.650
After sweep 8 energy=-14.470658408604907  maxlinkdim=72 maxerr=9.97E-12 time=0.599
After sweep 9 energy=-14.470658424893424  maxlinkdim=72 maxerr=9.91E-12 time=0.656
After sweep 10 energy=-14.470658427248509  maxlinkdim=68 maxerr=9.99E-12 time=0.656


In [9]:
H1 = Hamiltonian1(sites) 
ϕ = tdvp(
    H1,
    2.0im,
    psi0;
    time_step=1.0im,
    maxdim=30,
    cutoff=1e-10,
    normalize=true,
    reverse_step=false,
    outputlevel=1,
  )

After sweep 1: maxlinkdim=30 maxerr=6.77E-08 current_time=0.0 + 1.0im time=48.072
After sweep 2: maxlinkdim=30 maxerr=2.01E-07 current_time=0.0 + 2.0im time=4.836


MPS
[1] ((dim=2|id=650|"Link,l=1"), (dim=2|id=657|"S=1/2,Site,n=1"))
[2] ((dim=8|id=384|"Link,l=2"), (dim=4|id=315|"S=3/2,Site,n=2"), (dim=2|id=650|"Link,l=1"))
[3] ((dim=16|id=691|"Link,l=3"), (dim=2|id=397|"S=1/2,Site,n=3"), (dim=8|id=384|"Link,l=2"))
[4] ((dim=30|id=775|"Link,l=4"), (dim=4|id=296|"S=3/2,Site,n=4"), (dim=16|id=691|"Link,l=3"))
[5] ((dim=30|id=778|"Link,l=5"), (dim=2|id=692|"S=1/2,Site,n=5"), (dim=30|id=775|"Link,l=4"))
[6] ((dim=30|id=878|"Link,l=6"), (dim=4|id=230|"S=3/2,Site,n=6"), (dim=30|id=778|"Link,l=5"))
[7] ((dim=30|id=489|"Link,l=7"), (dim=2|id=727|"S=1/2,Site,n=7"), (dim=30|id=878|"Link,l=6"))
[8] ((dim=30|id=567|"Link,l=8"), (dim=4|id=169|"S=3/2,Site,n=8"), (dim=30|id=489|"Link,l=7"))
[9] ((dim=30|id=58|"Link,l=9"), (dim=2|id=505|"S=1/2,Site,n=9"), (dim=30|id=567|"Link,l=8"))
[10] ((dim=30|id=994|"Link,l=10"), (dim=4|id=177|"S=3/2,Site,n=10"), (dim=30|id=58|"Link,l=9"))
[11] ((dim=28|id=203|"Link,l=11"), (dim=2|id=959|"S=1/2,Site,n=11"), (dim=30|id=994|"Li

In [10]:
exp_Sz = expect( ϕ, "Sz" )

14-element Vector{Float64}:
 -0.358378203585628
  1.3612815462815353
 -0.3641414705919272
  1.362143056748451
 -0.3586592988347623
  1.3576715404661102
 -0.35778335980229675
  1.3576873183244433
 -0.3575142485894288
  1.357023689862273
 -0.35720825378667403
  1.357237261782382
 -0.35734809520148164
  1.3579801734775852

In [11]:
sum=0
for i in 1:length(exp_Sz)
    sum+=exp_Sz[i]
end

sum

6.999991656550581

In [12]:
using Plots
using LaTeXStrings
plot( expect(ϕ,"Sz"),marker=:circle )

GKS: dlopen(/Users/Hank/.julia/artifacts/66ebf8a541da7166677a6e2ac6b8277592d32044/lib/cairoplugin.so, 1): Library not loaded: @rpath/libcairo.2.dylib
  Referenced from: /Users/Hank/.julia/artifacts/66ebf8a541da7166677a6e2ac6b8277592d32044/lib/cairoplugin.so
  Reason: Incompatible library version: cairoplugin.so requires version 11603.0.0 or later, but libcairo.2.dylib provides version 2.0.0


In [13]:
# TDVP parameters
time_step = 1.0im
maxdim = 30
cutoff = 1e-10
normalize = true
reverse_step = false
outputlevel = 1


# Initialize lists to store total times and sums of expectation values
total_times = [1.0im + (i-1)*time_step for i in 1:15]
sum_values = []

for total_time in total_times
    # Evolve the state
    psi = tdvp(H1, total_time, psi0; time_step=time_step, maxdim=maxdim, cutoff=cutoff, normalize=normalize, reverse_step=reverse_step, outputlevel=outputlevel)
    
    # Compute the expectation value of Sz
    exp_Sz = expect(psi, "Sz")
    
    # Sum the expectation values
    sum=0
    
    for i in 1:length(exp_Sz)
        sum+=exp_Sz[i]
    end
    
    # Store the sum
    push!(sum_values, sum)
end


# Plot the sum of expectation values vs total time
plot(abs.(total_times), sum_values, xlabel="time", ylabel="< Sz>", lw=2, label="Sum(Sz)")



After sweep 1: maxlinkdim=30 maxerr=6.77E-08 current_time=0.0 + 1.0im time=5.992
After sweep 1: maxlinkdim=30 maxerr=6.77E-08 current_time=0.0 + 1.0im time=5.287
After sweep 2: maxlinkdim=30 maxerr=2.01E-07 current_time=0.0 + 2.0im time=4.041
After sweep 1: maxlinkdim=30 maxerr=6.77E-08 current_time=0.0 + 1.0im time=5.254
After sweep 2: maxlinkdim=30 maxerr=2.01E-07 current_time=0.0 + 2.0im time=4.484
After sweep 3: maxlinkdim=30 maxerr=3.83E-07 current_time=0.0 + 3.0im time=4.274
After sweep 1: maxlinkdim=30 maxerr=6.77E-08 current_time=0.0 + 1.0im time=5.261
After sweep 2: maxlinkdim=30 maxerr=2.01E-07 current_time=0.0 + 2.0im time=4.307
After sweep 3: maxlinkdim=30 maxerr=3.83E-07 current_time=0.0 + 3.0im time=4.632
After sweep 4: maxlinkdim=30 maxerr=3.99E-07 current_time=0.0 + 4.0im time=4.074
After sweep 1: maxlinkdim=30 maxerr=6.77E-08 current_time=0.0 + 1.0im time=5.241
After sweep 2: maxlinkdim=30 maxerr=2.01E-07 current_time=0.0 + 2.0im time=4.153
After sweep 3: maxlinkdim=30

After sweep 11: maxlinkdim=30 maxerr=1.61E-07 current_time=0.0 + 11.0im time=4.176
After sweep 12: maxlinkdim=30 maxerr=1.42E-07 current_time=0.0 + 12.0im time=4.718
After sweep 13: maxlinkdim=30 maxerr=1.19E-07 current_time=0.0 + 13.0im time=4.183
After sweep 14: maxlinkdim=30 maxerr=1.07E-07 current_time=0.0 + 14.0im time=4.251
After sweep 1: maxlinkdim=30 maxerr=6.77E-08 current_time=0.0 + 1.0im time=5.869
After sweep 2: maxlinkdim=30 maxerr=2.01E-07 current_time=0.0 + 2.0im time=4.473
After sweep 3: maxlinkdim=30 maxerr=3.83E-07 current_time=0.0 + 3.0im time=4.229
After sweep 4: maxlinkdim=30 maxerr=3.99E-07 current_time=0.0 + 4.0im time=4.279
After sweep 5: maxlinkdim=30 maxerr=3.93E-07 current_time=0.0 + 5.0im time=4.174
After sweep 6: maxlinkdim=30 maxerr=3.59E-07 current_time=0.0 + 6.0im time=4.875
After sweep 7: maxlinkdim=30 maxerr=3.13E-07 current_time=0.0 + 7.0im time=4.278
After sweep 8: maxlinkdim=30 maxerr=2.60E-07 current_time=0.0 + 8.0im time=4.202
After sweep 9: maxli

In [14]:
sum_values

15-element Vector{Any}:
 6.999998483990311
 6.999991656550581
 6.999982830170248
 6.999973166217292
 6.999968174164938
 6.9999691851035895
 6.999969499973815
 6.99997317052384
 6.999976573787698
 6.999978505432089
 6.999982847651388
 6.999984962541842
 6.99998729720072
 6.9999892166637
 6.99999031395158